# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
Humidity_df = pd.read_csv("../output_data/WeatherPy_Data.csv")
Humidity_df.head()


,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,6.80,78,90,12.66,CA,1619410536
1,1,cape town,-33.9258,18.4232,62.01,88,20,6.91,ZA,1619410500
2,2,saint george,37.1041,-113.5841,70.00,14,1,0.92,US,1619410570
3,3,busselton,-33.6500,115.3333,77.00,48,99,4.00,AU,1619410548
4,4,hithadhoo,-0.6000,73.0833,82.60,67,63,4.47,MV,1619410559


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = Humidity_df[["Lat", "Lng"]].astype(float)
Humidity = Humidity_df["Humidity"].astype(float)

# Create Heatmap layer
fig = gmaps.figure(center = [0,0],zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)
#Display Heat map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 92 degrees but higher than 85.
Ideal_Max_Temp=89
Ideal_Min_Temp=79
#Wind speed less than 10 mph.
Ideal_Wind_Speed=5
#cloudiness less than 5.
Ideal_Cloudiness=5



#Creat new dataframe
Narrow_Weather_df = Humidity_df.iloc[:,1:]
Narrow_Weather_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,yellowknife,62.4560,-114.3525,6.80,78,90,12.66,CA,1619410536
1,cape town,-33.9258,18.4232,62.01,88,20,6.91,ZA,1619410500
2,saint george,37.1041,-113.5841,70.00,14,1,0.92,US,1619410570
3,busselton,-33.6500,115.3333,77.00,48,99,4.00,AU,1619410548
4,hithadhoo,-0.6000,73.0833,82.60,67,63,4.47,MV,1619410559


In [13]:
#created narrowed down df 
Narrow_Weather_df = Narrow_Weather_df.loc[(Narrow_Weather_df["Max_Temp"] < Ideal_Max_Temp) &
                                         (Narrow_Weather_df["Max_Temp"] > Ideal_Min_Temp) & 
                                         (Narrow_Weather_df["Wind_Speed"] < Ideal_Wind_Speed) &
                                         (Narrow_Weather_df["Cloudiness"] < Ideal_Cloudiness)] 
                                         
Narrow_Weather_df.head()


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
100,singaraja,-8.1120,115.0882,86.58,60,0,3.27,ID,1619410636
483,fare,-16.7000,-151.0167,81.50,63,2,1.25,PF,1619410415
499,bodden town,19.2833,-81.2500,80.60,78,0,2.30,KY,1619410728
518,karachi,24.9056,67.0822,86.00,66,0,4.61,PK,1619410675


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#added column on DF "Hotel Name"
hotel_df = Narrow_Weather_df
hotel_df["Hotel_Name"]= ""
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
100,singaraja,-8.1120,115.0882,86.58,60,0,3.27,ID,1619410636,
483,fare,-16.7000,-151.0167,81.50,63,2,1.25,PF,1619410415,
499,bodden town,19.2833,-81.2500,80.60,78,0,2.30,KY,1619410728,
518,karachi,24.9056,67.0822,86.00,66,0,4.61,PK,1619410675,


In [15]:
#base urls and paramater to pull request
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": "",  
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    City_Name = row['City']
    Lat = row['Lat']
    Lng = row['Lng']
    params["location"]= f"{Lat},{Lng}"
    print(f"Retrieving Results for Index {index}: {City_Name} Location {Lat},{Lng}")
    response = requests.get(base_url,params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel with-in 5000 meters of {City_Name} is {results[0]['name']} ")
        hotel_df.loc[index,"Hotel_Name"]=results[0]['name']
    
    
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print(f"------------")



Retrieving Results for Index 100: singaraja Location -8.112,115.0882
Closest hotel with-in 5000 meters of singaraja is Puri Bagus Lovina Resort 
------------
Retrieving Results for Index 483: fare Location -16.7,-151.0167
Closest hotel with-in 5000 meters of fare is Maitai Lapita Village Huahine 
------------
Retrieving Results for Index 499: bodden town Location 19.2833,-81.25
Closest hotel with-in 5000 meters of bodden town is Will T Place 
------------
Retrieving Results for Index 518: karachi Location 24.9056,67.0822
Closest hotel with-in 5000 meters of karachi is Sadaat 
------------


In [9]:
hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
100,singaraja,-8.1120,115.0882,86.58,60,0,3.27,ID,1619410636,Puri Bagus Lovina Resort
483,fare,-16.7000,-151.0167,81.50,63,2,1.25,PF,1619410415,Maitai Lapita Village Huahine
499,bodden town,19.2833,-81.2500,80.60,78,0,2.30,KY,1619410728,Will T Place
518,karachi,24.9056,67.0822,86.00,66,0,4.61,PK,1619410675,Sadaat


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

In [38]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color="black",
    stroke_color="black",scale=3,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))